In [7]:
#随机更换代理，是反爬的trick之一，但是在实践过程中，并不work；
#更换header也是，但是从爬取的实践经验来看，time.sleep()的设置是非常重要的；
#sleep的时间设置长些，也就意味着访问url的间隔时间会长些，这样可以有效避免反爬；
import re
import requests
import urllib
import sys
!{sys.executable} -m pip install bs4
from bs4 import BeautifulSoup
import random
!{sys.executable} -m pip install pandas
import pandas as pd
import csv
from itertools import cycle
import time
import numpy as np
import random
!{sys.executable} -m pip install tqdm
import tqdm

def get_data(page, HEADER):
    #一页有30个帖子
    basic_url = 'https://www.douban.com/group/634189/discussion?start=%d'%(page*30)
    #basic_url = 'https://www.douban.com/group/638298/discussion?start=0'
    response = requests.get(basic_url, headers = HEADER, timeout= 60)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    
    #获取帖子中的title,name,response,time;
    data = soup.find_all('tr')
    #filtered_data[i] = ['精品', 'title','name','response_nums','time'] or ['title','name','response_nums','time']
    filtered_data = [[i for i in i.text.strip().split('\n')if i.strip() != ''] for i in data[1:]]
    
    #获取帖子中的topic's url及发帖人id；
    topic_people_urls = [url.get('href') for url in soup.find_all('a')]
    topic_url = []
    people_url = []
    for topic_people_url in topic_people_urls:
        if topic_people_url is not None:
            topic_people_url_ = topic_people_url.split('/')
            #topic_url in one page;
            if len(topic_people_url_) < 3:
                continue
            if topic_people_url_[-3] == 'topic':
                topic_url.append(topic_people_url)
            #people_url in one page;
            elif topic_people_url_[-3] == 'people':
                people_url.append(topic_people_url)
    
             
    return filtered_data, topic_url, people_url

def get_most_popular_cont(soup):
    dataOfMostPopular = soup.find_all('ul', "topic-reply popular-bd")[0]

    #最赞回应的发帖方式（电脑orApp）；
    waysToPostsInMostPopular = []
    for i in dataOfMostPopular.find_all('li'):
        try:
            wayToPost = i.find_all('span', 'via')[0].text
        except:
            wayToPost = ''
        waysToPostsInMostPopular.append(wayToPost)

    #最赞回应的发帖内容；
    contsOfMostPopular = [i.text for i in dataOfMostPopular.find_all('p')]

    #最赞回应用户的nickname；
    nickNamesOfMostPopular = [i.text.split('\n')[1] for i in dataOfMostPopular.find_all('h4')]

    #最赞回应用户的id；
    idsOfMostPopular = [i.find('a').get('href') for i in dataOfMostPopular.find_all('h4')]
    
    return waysToPostsInMostPopular, contsOfMostPopular, nickNamesOfMostPopular, idsOfMostPopular


def get_way_id_name_cont(soup):
    data = soup.find_all('div', 'reply-doc content')
    #发帖方式（电脑orApp）
    ways = []
    for i in data:
        try:
            way = i.find_all('span', 'via')[0].text
        except:
            way = ''
        ways.append(way)

    #回复内容；
    conts = [j.text for i in data for j in i.find_all('p')]

    #回复人的nickname;
    nickNames = [i.find_all('a')[0].text for i in data]
    
    #回复人的id
    ids = [i.find_all('a')[0].get('href') for i in data]
    
    return ways, conts, nickNames, ids


def get_content(topic_url, custom):
    
    allWaysOfReplyContents = []
    allContsOfReplyContents = []
    allNicknamesOfReplyContents = []
    allIdsOfReplyContents = []
    allWaysOfReplyContents_ = []
    allContsOfReplyContents_ = []
    allNicknamesOfReplyContents_ = []
    allIdsOfReplyContents_ = []
    imgNumsInTopic_ = []
    topicDocs_ = []
    topicWays_ = []
    most_popular = []
    
    for url in topic_url:
        HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
        response = requests.get(url, headers=HEADER, timeout= 60)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        
        #帖子中包含图片的数量；
        try:
            imgNumsInTopic = len(soup.find_all('div', "rich-content topic-richtext")[0].find_all('img'))
        except:
            imgNumsInTopic = 0
        imgNumsInTopic_.append(imgNumsInTopic)
        
        topic = [i for i in soup.find_all('div','topic-doc')[0].text.split('\n') if (i != '' and len(soup.find_all('div','topic-doc')) > 0)]
        
        try:
            topicDocs = ''.join([i.text for i in soup.find_all('div', "rich-content topic-richtext")[0].find_all('p')])
        except:
            topicDocs = '出现list_index错误'
        topicWays = topic[-1]
        
        topicDocs_.append(topicDocs)
        topicWays_.append(topicWays)
        
        if custom == 'mostPopular':
            #最赞回应的发帖方式（电脑orApp）、内容、昵称及id；
            waysToPostsInMostPopular, contsOfMostPopular, nickNamesOfMostPopular, idsOfMostPopular = get_most_popular_cont(soup)
            allWaysOfReplyContents_.append(waysToPostsInMostPopular)
            allContsOfReplyContents_.append(contsOfMostPopular)
            allNicknamesOfReplyContents_.append(nickNamesOfMostPopular)
            allIdsOfReplyContents_.append(idsOfMostPopular)
        
        elif custom == '1stPage':
            #第一页的发帖方式（电脑orApp）、内容、昵称及id；
            waysOfReplyIn1stPage, contsOfReplyIn1stPage, nickNamesOfReplyIn1stPage, idsOfReplyIn1stPage = get_way_id_name_cont(soup)
            try:
                len_popular, _, _, _ = get_most_popular_cont(soup)
                index = len(len_popular)
            except:
                index = 'none'
            allWaysOfReplyContents_.append(waysOfReplyIn1stPage)
            allContsOfReplyContents_.append(contsOfReplyIn1stPage)
            allNicknamesOfReplyContents_.append(nickNamesOfReplyIn1stPage)
            allIdsOfReplyContents_.append(idsOfReplyIn1stPage)
            most_popular.append(index)
            
        elif custom == 'allPages':
            #全部页的发帖方式（电脑orApp）、内容、昵称及id；
#             waysOfReplyIn1stPage, contsOfReplyIn1stPage, nickNamesOfReplyIn1stPage, idsOfReplyIn1stPage = get_way_id_name_cont(soup)
#             allWaysOfReplyContents.extend(waysOfReplyIn1stPage)
#             allContsOfReplyContents.extend(contsOfReplyIn1stPage)
#             allNicknamesOfReplyContents.extend(nickNamesOfReplyIn1stPage)
#             allIdsOfReplyContents.extend(idsOfReplyIn1stPage)
            # TODO: 写一个for loop取所有page的url
            allPagesOfReplyUrls = [url]
            getNextReplyPageUrl(url, allPagesOfReplyUrls)
            for page in allPagesOfReplyUrls:
                response = requests.get(page, headers = HEADER, timeout= 60)
                htmlOfPage = response.text
                soupOfPage = BeautifulSoup(html, 'lxml')
                ways, conts, nickNames, ids = get_way_id_name_cont(soupOfPage)
                allWaysOfReplyContents.extend(ways)
                allContsOfReplyContents.extend(conts)
                allNicknamesOfReplyContents.extend(nickNames)
                allIdsOfReplyContents.extend(ids)
                time.sleep(3)
            
            allWaysOfReplyContents_.append(allWaysOfReplyContents)
            allContsOfReplyContents_.append(allContsOfReplyContents)
            allNicknamesOfReplyContents_.append(allNicknamesOfReplyContents)
            allIdsOfReplyContents_.append(allIdsOfReplyContents)
            
        time.sleep(3)
            
    return imgNumsInTopic_, topicDocs_, topicWays_, allWaysOfReplyContents_, allContsOfReplyContents_, allNicknamesOfReplyContents_, allIdsOfReplyContents_, most_popular

def getNextReplyPageUrl(url, res):
    '''

    :return:
    '''
    if url==None or len(url)==0:
        return
    try:
        response = requests.get(url, headers = HEADER, timeout= 60)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        data = soup.findAll('span', 'next')
        # 帖子只有一页
        if len(data) == 0:
            return
        new_url = [i.find('a').get('href') for i in data if i.find('a') is not None]
        if len(new_url) == 0:
            return
        res.append(new_url[0])
        getNextReplyPageUrl(new_url[0], res)
    except Exception as e:
        print("get attr error!", e)

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [8]:
from tqdm import tqdm
import traceback
import logging
HEADERS_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
        ]

count = 1
# NOTE: 在这里改你想要爬那几页，range(a, b)表示爬第a页到第(b - 1)页的， 比如下面的range(0, 10)就是爬第0页到第9页(计算机计数一般从0开始记而不是从1)
for page in tqdm(range(0, 1)):
    
    if count % 18 == 0:
        time.sleep(random.randint(220, 230))
    
    if count % 37 == 0:
        time.sleep(random.randint(265, 280))
    
    if count % 56 == 0:
        time.sleep(random.randint(303, 336))
    
    HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
    
    s_time = time.time()
    
    filtered_data, topic_url, people_url = get_data(page, HEADER)
  
    #topic_url = [i for i in topic_url if i.split(':')[0] != 'https']
    for i, j in enumerate(topic_url):
        if j.split(':')[0] != 'https':
            topic_url[i] = topic_url[-1]   
    time.sleep(5)
    
    # NOTE: 如果只想爬第一页回复，就把get_content函数的第2个参数设为'1stPage', 如果想爬所有页的回复，把get_content函数的第二个参数设为'allPages'
    imgNumsInTopic_, topicDocs_, topicWays_, allWaysOfReplyContents_, allContsOfReplyContents_, allNicknamesOfReplyContents_, allIdsOfReplyContents_, most_popular = get_content(topic_url, '1stPage')
    
    jingpin = []
    for i in filtered_data:
        try:
            jingpin.append(i[-5])
        except:
            jingpin.append('')
    
    try:
        titles = [i[-4] for i in filtered_data]
    except:
        titles = [i[0] for i in filtered_data]
    try:
        names = [i[-3] for i in filtered_data]
        responses_num = [i[-2] for i in filtered_data]
        times = [i[-1] for i in filtered_data]

        #if len(allWaysOfReplyContents_) != len(titles):
         #   allWaysOfReplyContents_.append(allWaysOfReplyContents_[-1])

        csv = pd.DataFrame({'title':titles, 'name':names, 'id':people_url,
                            'responses_num':responses_num, 'time':times, 'docs':topicDocs_,
                            'reply_content':allContsOfReplyContents_, 'img_nums':imgNumsInTopic_, 
                            'topic_url':topic_url, 'reply_id':allIdsOfReplyContents_, 'jinpin':jingpin, 
                            'popular_index':most_popular, 'topic_ways':topicWays_, 
                            'replyWays':allWaysOfReplyContents_})
        # NOTE: 1) 这里面输出的csv文件名, e.g. rencai.csv, 要和data_vis.ipynb里读入的一致. 
        # 2) 写文件的时候是在末尾append, 如果你强制终止了程序，或者程序中间挂了，为了避免某些页重复被爬从而重复统计，
        # 要么把这个输出文件删了重新跑程序， 要么在上面我备注的改爬取页数的地方，修改将要爬取的页，从上次停止的地方继续.
        csv.to_csv(r'rencai.csv', mode= 'a', header= True, encoding= 'utf_8_sig')
        time.sleep(random.randrange(6, 8, 1))
        e_time = time.time()
        log = '第%d页'%(page+1) + '||' + '发帖时间：%s'%times[-1] + '||' + '耗时：%f秒'%(e_time-s_time)
        print(log)
        count = count + 1
    except IOError:
        print("I/O error")
    except IndexError:
        logging.error(traceback.format_exc())
    except Exception as e:
        logging.error(traceback.format_exc())
        raise

  0%|          | 0/1 [00:00<?, ?it/s]

检测到有异常请求从你的 IP 发出，请 <a href="https://accounts.douban.com/passport/login?redir=https%3A%2F%2Fwww.douban.com%2Fgroup%2F634189%2Fdiscussion%3Fstart%3D0">登录</a> 使用豆瓣。



  0%|          | 0/1 [00:09<?, ?it/s]


KeyboardInterrupt: 